## VISUALIZATION PROJECT Geospatial Business Intelligence (BI)


In [29]:
#Tras iniciar servidor de DB mongo, importo módulo 'pymongo'.
import pymongo
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize
#Asigno variable al cliente de DB y a la DB.
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [30]:
#Verifico que se han importado todos los documentos.
cursor = db.companies.find()
data = list(cursor)
#print(len(data))

In [31]:
def q_qty_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return len(list(map(lambda r:r,list(q))))
atr1 = 'category_code'
var1 = 'web'
#print(q_qty_1(atr1,var1))

In [32]:
def q_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return list(map(lambda r:[(r['name']),(r['offices'])],list(q)))
atr1 = 'category_code'
var1 = 'web'
#display(q_1(atr1,var1))

In [33]:
#Creo dataframe con los datos
df = pd.DataFrame(data)
#display(df.dtypes)
columns = df.dtypes
#print(columns)

In [34]:
#Extraigo columnas para tratamiento (para no copiarlas a mano)
#g = df.columns.to_series().groupby(df.dtypes).groups
#{k.name: v for k, v in g.items()}

In [35]:
#Clasificación de columnas originales para limpieza del dataframe
col_float = ['deadpooled_day','deadpooled_month','deadpooled_year','founded_day','founded_month',
             'founded_year','number_of_employees']
col_obj = ['_id','acquisition','acquisitions','alias_list','blog_feed_url','blog_url','category_code',
           'competitions','created_at','crunchbase_url','deadpooled_url','description','email_address',
        'external_links','funding_rounds','homepage_url','image','investments','ipo','milestones', 
           'name','offices','overview','partners','permalink','phone_number','products','providerships',
        'relationships','screenshots','tag_list','total_money_raised','twitter_username','updated_at',
           'video_embeds']
col_drop1 = ['deadpooled_day','deadpooled_month','founded_day','founded_month','alias_list','blog_feed_url',
             'blog_url','created_at','crunchbase_url','deadpooled_url','email_address','external_links',
            'image','permalink','phone_number','screenshots','tag_list','updated_at','video_embeds']
col_drop2 = ['acquisition','acquisitions','competitions','description','funding_rounds','homepage_url',
             'investments','ipo','milestones','partners','products','providerships','relationships',
             'twitter_username','overview',]
col_ok = ['deadpooled_year','founded_year','number_of_employees','_id','category_code','name','offices',
          'total_money_raised']

print(len(col_float)+len(col_obj),len(col_drop1)+len(col_drop2)+len(col_ok))

42 42


In [52]:
#Elimino columnas con valores que no me interesan
df_drop1 = df.drop(col_drop1, axis=1)
df_drop2 = df_drop1.drop(col_drop2, axis=1)
#Elijo solo las empresas con deadpooled_year == null
df_deadnull = df_drop2[df_drop2['deadpooled_year'].isnull()]
#Elimino filas con valores nulos de las columnas que me interesan
df_nulls = df_deadnull.dropna(subset=['founded_year','number_of_employees','name','offices',
          'total_money_raised'])
display(len(df_nulls))
#display(df_nulls.head())

7934

In [53]:
#Data frames por tipo de empresas y elimino las columnas _id y deadpooled_year 
col_cat = ['games_video','web','software']
df_cat = df_nulls[df_nulls['category_code'].isin(col_cat)]\
.drop(['_id','deadpooled_year'], axis=1)#.sort_values(by=['category_code','number_of_employees'])
print(len(df_cat))
#display(df_cat.head())

3752


In [38]:
#Defino funciones para la modificación de los datos de las columnas a analizar.

#Transforma un dato tipo float a uno tipo int.
def float_to_int(flt):
    integer = int(flt)
    return integer

#Transforma un dato tipo str en int devolviendo solo el número.
def str_to_int(strg):
    string = re.findall('\d+', strg )
    integer = int(string[0])*1000
    return integer


In [55]:
#Elimino registros irrelevantes (tomando solo los registros que me interesan)
df_regout = df_cat.copy()
df_regout['total_money_raised'] = df_regout['total_money_raised'].apply(str_to_int)
df_regout['founded_year'] = df_regout['founded_year'].apply(float_to_int)
#Elimino registros según criterios 
df_regout = df_regout[df_regout.total_money_raised != 0]
df_regout = df_regout[df_regout.number_of_employees != 0]
df_regout = df_regout[df_regout['offices'].map(len) > 0]
print(len(df_regout))
#display(df_regout.head())

768


In [40]:
#Verifico estructura del diccionario
#print(df_regout['offices']).iloc[100])

In [41]:
#d = {'type': 'Point', 'coordinates': [None, None]}
##s = d['coordinates']
#lon = d['coordinates'][0]
#if lon == None:
#    print('No hay valor de longitud para este registro')

In [42]:
#Transforma los datos de ubicación de las oficinas en formato GeoJson y selecciona solo la primera ubicación
def nested_to_list(dicts):
    lat_lon = [{"type":"Point","coordinates":[l['longitude'],l['latitude']]} for l in dicts]
    lst = lat_lon[0]
    long = lst['coordinates'][0]
    if long != None:
        return lst
    else:
        return 0

#print(nested_to_list([{'description': 'New York Office', 'address1': '', 'address2': '', 'zip_code': '', 'city': 'New York', 'state_code': None, 'country_code': 'USA', 'latitude': 40.756054, 'longitude': -73.986951},{'description': 'Barcelona Headquarters', 'address1': '', 'address2': '', 'zip_code': '', 'city': 'Barcelona', 'state_code': None, 'country_code': 'ESP', 'latitude': 41.387917, 'longitude': 2.1699187}]))

In [43]:
#Transformo valores de columnas aplicando las funciones anteriormente definidas.
df_colout = df_regout.copy()
df_colout['offices'] = df_colout['offices'].apply(nested_to_list)
print(len(df_colout))
#display(df_colout.head())

768


In [44]:
#print(df_colout['offices'])

In [45]:
#Elimino valores de GeoJson nulos
df_geoout = df_colout.copy()
df_geoout = df_geoout[df_geoout.offices != 0]
print(len(df_geoout))
display(df_geoout.head())

603


,category_code,founded_year,name,number_of_employees,offices,total_money_raised
6,web,2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",16000
20,web,2002,Plaxo,50.0,"{'type': 'Point', 'coordinates': [-122.055197,...",28000
23,web,1995,eBay,15000.0,"{'type': 'Point', 'coordinates': [-121.930035,...",6000
28,web,2007,Mahalo,40.0,"{'type': 'Point', 'coordinates': [-118.487267,...",21000
31,games_video,2006,Kyte,40.0,"{'type': 'Point', 'coordinates': [-122.409173,...",23000


In [46]:
#print(df_geoout['offices'])

In [47]:
df_geoout.to_json('prueba.json', orient="records", lines=True)

In [56]:
def concentration(company):
    cursor_near = db.fase1.find({
      "offices": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [company['offices']['coordinates'][0], company['offices']['coordinates'][1]]
          },
          "$minDistance": 0,
          "$maxDistance":9000
        }
      }
    })
    return list(cursor_near)

In [57]:
#Hago loop sobre las empresas
cursor_geojson = db.fase1.find()
clusters = []
for element in cursor_geojson:
    clusters.append(len(concentration(element)))


display(len(clusters))

2305

In [58]:
print(max(clusters))
#print(clusters)


132


In [51]:
target = list(db.fase1.find())
targets = []
for i in range(len(clusters)):
    if clusters[i] == max(clusters):
        targets.append([target[i]['offices']['coordinates'][0], target[i]['offices']['coordinates'][1]])

print(targets)

[[-122.457454, 37.79851]]


## NOTAS

In [ ]:
#df_off = json_normalize(df_cat, 'offices')
#display(df_off.head())

In [ ]:
df_fase1 = pd.DataFrame(list(cursor))
print(len(df_fase1))
display(df_fase1.head())

In [ ]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

In [ ]:
#Esto es para buscar con regex de mongo
#{ name: { $in: [ /^key./i ] } }    { tag_list: { $in: [ /.games./i ] } }  
#{ updated_at: { $in: [ /2008$/ ] } } 
#Para descartar arrays vacios en mongo
#{offices:{ $gt: [] } } (hay otras opciones en un link en pendientes)

In [ ]:
def convertRestaurant(restaurant):
    return {
        "name": restaurant["name"],
        "cuisine":restaurant["cuisine"],
        "position": toGeoJSON(restaurant["address"]["coord"])
    }


In [ ]:
#Elimino rows que no me aportan nada
#df_values = df_nulls[df_nulls['offices'].map(len) > 0]
#df_values = df_values[df_values.number_of_employees != 0]
#df_values = 
#print(len(df_values))
#display(df_values.head(10))